In [1]:
import collections
import random
import re
from collections import Counter
from itertools import islice

### **Make Preprocess for Text**

In [2]:
arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [3]:
def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text
print(remove_diacritics("الْعَرَبِيّةُ"))

العربية


In [4]:
def remove_repeating_char(text):
    if 'الله' in text:
        return re.sub(r'(.)\1{2,}', r'\1\1', text)  # keep 2 repeats if 'الله' is present
    else:
        return re.sub(r'(.)\1+', r'\1', text)  # keep only 1 repeat otherwise

print(remove_repeating_char("الوووو ابرار"))
print(remove_repeating_char("الله"))

الو ابرار
الله


In [5]:
import string
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

print(remove_punctuations('هذه نص باللغة العربية! تحتوي على علامات الترقيم؛ وهي: ،.؟'))

هذه نص باللغة العربية تحتوي على علامات الترقيم وهي 


In [6]:
def remove_repeating_spaces(text):
    return re.sub(' +', ' ', text)

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_stop_words(text):
    arabic_stop_words = set(stopwords.words('arabic'))
    words = word_tokenize(text)
    return ' '.join([w for w in words if w not in arabic_stop_words])

In [8]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

print(normalize_arabic(".إنّ الذين يتّبعون الصّيغ المعمّاة للكتابة، يتجنّبون الكلمات التي تتضمّن حروفاً مهمّلة."))

.انّ الذين يتّبعون الصّيغ المعمّاه للكتابه، يتجنّبون الكلمات التي تتضمّن حروفاً مهمّله.


In [9]:
def process_text(text):
    clean_text = remove_diacritics(text)
    clean_text = remove_repeating_char(clean_text)
    clean_text = remove_punctuations(clean_text)
    clean_text = remove_repeating_spaces(clean_text)
    clean_text = remove_stop_words(clean_text)
    clean_text = normalize_arabic(clean_text)
    
    return clean_text.split()
    
print('result:',process_text("العربية الفصحى اجمل من اي لغه اخري."))

result: ['العربيه', 'الفصحي', 'اجمل', 'اي', 'لغه', 'اخري']


#### ***Check if text exists in document*** 

In [10]:
def document_features(document, corpus_features):
    document_words = set(document) # create a set of unique words in the document
    # print('document_words:',document_words)
    features = {}
    for word in corpus_features:
        features['contains({})'.format(word)] = (word in document_words)
        # print('features:',features)
    return features

print(document_features("إخراج الكلمات المميزه الغير مكرره مكرره".split(), 
                        ["اجمل","الكلمات", "المميزه", "الغير", "مكرره"]))

{'contains(اجمل)': False, 'contains(الكلمات)': True, 'contains(المميزه)': True, 'contains(الغير)': True, 'contains(مكرره)': True}


### ***Main***

In [11]:
all_features = list()
texts = list()
data_labels = list()

positive_file = 'datasets/positive_tweets_arabic_20181207_300.txt'
negative_file = 'datasets/negative_tweets_arabic_20181207_300.txt'

min_freq = 13

##### ***Read Pos & Neg files***

In [12]:
with open(positive_file, encoding='utf-8') as tweets_file:
    for line in tweets_file:
        text_features = process_text(line)
        all_features += text_features
        texts.append(text_features)
        data_labels.append('pos')

with open(negative_file, encoding='utf-8') as tweets_file:
    for line in tweets_file:
        text_features = process_text(line)
        all_features += text_features
        texts.append(text_features)
        data_labels.append('neg')


all_features: ['بالضبط', '💓', '💝', '💝', '💗', '💖', '💖', '💗', '💝', '💓', '💕', '💞', 'حظ', 'سلسال', 'بيدك', 'تلوي', '💕', 'مرحبا', '💓', '💓', '💓', '💓', '💓', '💋', '💋', '❤', '❤', '❤', '❤', '💕', '💕', '💕', '💕', '💕', '👀', '👀', '😙', '😙', '😙', '😙', '😙', '😙', '😙', 'تفن', 'صنع', 'قهوتك', '🎼', 'وابتكر', 'لنفسك', 'يوميا', 'لطيفا', 'واغلق', 'الابواب', 'التي', 'تفسد', 'مزاجك', '☕', '🎼', '🕊', '💕', '🌹', 'يارب', 'سفرني', 'اليابان', '💕', '🍃', '🌹', '🍃', '﴿', 'صل', 'عليهم', 'صلاتك', 'سكن', '﴾', 'كانت', 'صلاته', 'ﷺ', 'لمسلمين', 'سكن', 'وطمانينه', 'فكيف', 'بصلاه', 'ﷲ', 'ﷻ', 'مرات', 'صليت', 'حب', 'هالوقت', 'مو', 'ثوبي', '💕', 'رويت', 'واعملك', 'فوت', 'الي', 'بعدك', '💕', '🌸', 'اللهم', 'صلي', 'وسلم', '💕', 'اللهم', 'صل', 'وسلم', 'وبارك', 'نبينا', 'وحبيبنا', 'وقدوتنا', 'محمد', 'وعلي', 'اله', 'وصحبه', 'اجمعين', '💕', 'الله', 'صل', 'وسل', 'وب', 'ارك', 'نبي', 'وحبيبنا', 'حمد', '💕', 'اللهم', 'صل', 'وسلم', 'نبينا', 'محمد', '💕', '💕', 'رحمات', 'يهبها', 'الله', 'ان', 'تشعر', 'فقط', 'تخلي', 'حولك', 'وقوتك', 'والتجا', 'الي', 'حول

In [13]:
print('data size', len(data_labels))
print('# of positive', data_labels.count('pos'))
print('# of negative', data_labels.count('neg'))

data size 21083
# of positive 6748
# of negative 14335


##### ***Remove Frequence*** 

In [14]:
import nltk
from nltk.metrics.scores import f_measure

tweets = [(t, l) for t, l in zip(texts, data_labels)]
print('tweets', tweets[:5])
random.shuffle(tweets)
print('sample tweets')
for t in tweets[:10]: print(t)  # see the first 10 instances
print('\nall features', all_features[:20])
print('len(all_features):', len(all_features))

all_features_freq = nltk.FreqDist(w for w in all_features)
print(all_features_freq)
print('\nmost common features:', all_features_freq.most_common(20))
print('\nfreq of في: ', all_features_freq['في'])
thr = min_freq / len(all_features)
print('The minimum frequency threshold for features:', thr)

tweets [(['بالضبط', '💓', '💝', '💝', '💗', '💖', '💖', '💗', '💝', '💓', '💕', '💞'], 'pos'), (['حظ', 'سلسال', 'بيدك', 'تلوي', '💕'], 'pos'), (['مرحبا', '💓', '💓', '💓', '💓', '💓', '💋', '💋', '❤', '❤', '❤', '❤', '💕', '💕', '💕', '💕', '💕', '👀', '👀', '😙', '😙', '😙', '😙', '😙', '😙', '😙'], 'pos'), (['تفن', 'صنع', 'قهوتك', '🎼', 'وابتكر', 'لنفسك', 'يوميا', 'لطيفا', 'واغلق', 'الابواب', 'التي', 'تفسد', 'مزاجك', '☕', '🎼', '🕊', '💕', '🌹'], 'pos'), (['يارب', 'سفرني', 'اليابان', '💕'], 'pos')]
sample tweets
(['عروض', 'تصل', 'الي', '٪', 'بالاضافه', 'الي', 'كوبون', 'خصم', '٪', 'المنتجات', 'المخفضه', '😙', '😋', '😆', '😍', '😍', '😍', '😍', 'كوبون', 'كود', 'خصم'], 'pos')
(['صديقه', 'الايام', 'الخوالي', 'وباصات', 'المدرسه', '😍', '😍', '♥', '️', '😭', 'ميرسي', 'جميله', '’', '😉', '😘'], 'pos')
(['االله', '♥', 'ارح', 'قلوبا', 'انهكها', 'التمن', 'بشر', 'اصحابها', 'بفرح', 'يذكرهم', 'بوجعهم'], 'neg')
(['ناوي', 'يحتلني', 'احتلال', 'رسمي', 'طريق', 'الرابط', '😝', '☹', '️', '😒', '😂', '😂'], 'neg')
(['عروض', 'تصل', 'الي', '٪', 'بالاضافه', 'ال

In [24]:
my_features = set([word for word in all_features if all_features_freq.freq(word) > thr])
print(len(my_features), 'are kept out of', len(all_features))
print('features are selected')
print('------------------------------------')

print('sample features:')
print(list(my_features)[:10])
print('------------------------------------')

print('generating features for documents ...')
feature_sets = [(document_features(d, my_features), c) for (d, c) in tweets]
print('feature_sets', feature_sets[:3])


2665 are kept out of 267647
features are selected
------------------------------------
sample features:
['وحده', 'مت', 'ورشيق', 'يسوي', 'الاسعار', 'شوفو', 'عدد', 'وكانهم', 'لوحه', 'لاه']
------------------------------------
generating features for documents ...
feature_sets [({'contains(وحده)': False, 'contains(مت)': False, 'contains(ورشيق)': False, 'contains(يسوي)': False, 'contains(الاسعار)': False, 'contains(شوفو)': False, 'contains(عدد)': False, 'contains(وكانهم)': False, 'contains(لوحه)': False, 'contains(لاه)': False, 'contains(جده)': False, 'contains(استخراج)': False, 'contains(عنا)': False, 'contains(😖)': False, 'contains(واحنا)': False, 'contains(كنت)': False, 'contains(تروح)': False, 'contains(الشمس)': False, 'contains(كميه)': False, 'contains(مباراه)': False, 'contains(صلو)': False, 'contains(مالي)': False, 'contains(الصحه)': False, 'contains(ندي)': False, 'contains(الزمالك)': False, 'contains(شهر)': False, 'contains(قطارات)': False, 'contains(شعري)': False, 'contains(منياتي

##### ***Train & Test with Naive Bayes***

In [16]:
print('splitting documents into train and test ...')
print('data set size', len(data_labels))
train_percentage = 0.8
splitIndex = int(len(tweets) * train_percentage)
print('split index:', splitIndex)

# random.shuffle(feature_sets)
train_set, test_set = feature_sets[:splitIndex], feature_sets[splitIndex:]
y_train = [l for t, l in train_set]
y_test = [l for t, l in test_set]

print('data set:', Counter(data_labels))
print('train:', Counter(y_train))
print('test:', Counter(y_test))

print('training NaiveBayes classifier ...')
classifier = nltk.NaiveBayesClassifier.train(train_set)
print('training is done')

ref_sets = collections.defaultdict(set)
test_sets = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_set):
    ref_sets[label].add(i)
    observed = classifier.classify(feats)
    test_sets[observed].add(i)

print('accuracy:', nltk.classify.accuracy(classifier, test_set))
print('positive f-score:', f_measure(ref_sets['pos'], test_sets['pos']))
print('negative f-score:', f_measure(ref_sets['neg'], test_sets['neg']))


classifier.show_most_informative_features(20)

splitting documents into train and test ...
data set size 21083
split index: 16866
data set: Counter({'neg': 14335, 'pos': 6748})
train: Counter({'neg': 11445, 'pos': 5421})
test: Counter({'neg': 2890, 'pos': 1327})
training NaiveBayes classifier ...
training is done
accuracy: 0.8769267251600664
positive f-score: 0.8189745378444366
negative f-score: 0.9067720495778694
Most Informative Features
             contains(😙) = True              pos : neg    =    487.6 : 1.0
             contains(😚) = True              pos : neg    =    297.7 : 1.0
       contains(المخفضه) = True              pos : neg    =    250.4 : 1.0
             contains(⛔) = True              neg : pos    =    151.7 : 1.0
         contains(التزم) = True              neg : pos    =    150.2 : 1.0
           contains(نشر) = True              neg : pos    =    133.4 : 1.0
             contains(😀) = True              pos : neg    =    110.0 : 1.0
             contains(😆) = True              pos : neg    =    106.1 : 1.0
   

### ***Testing***

In [17]:
def analyze_arabic_text(text, classifier, features):
    processed_text = process_text(text)
    document_feats = document_features(processed_text, features)
    label = classifier.classify(document_feats)
    return label

In [18]:
# Example usage:
input_text = "أحببت هذا الفيلم كثيراً!💕"
predicted_label = analyze_arabic_text(input_text, classifier, my_features)
print("Predicted label:", predicted_label)

Predicted label: pos


In [19]:
# Example usage:
input_text = "اكره هذه الكليه"
predicted_label = analyze_arabic_text(input_text, classifier, my_features)
print("Predicted label:", predicted_label)

Predicted label: neg


In [20]:
# Example usage:
input_text = "😢انا حزينه جدا"
predicted_label = analyze_arabic_text(input_text, classifier, my_features)
print("Predicted label:", predicted_label)

Predicted label: neg


In [21]:
input_text = input("Enter a text: ")
predicted_label = analyze_arabic_text(input_text, classifier, my_features)
print('input text:', input_text, '\n')
print("Predicted label:", predicted_label)

input text: الحمد لله الشكر لك يا رب 

Predicted label: pos
